In [ ]:
import os
import numpy as np
import time
import pickle
from sklearn.metrics import confusion_matrix
%run Kernel.ipynb
%run DataSplitter.ipynb
%run Model_w_Conv.ipynb


class Trainer_conv:
    def __init__(self, data, label_pos, r, v, delta, beta):
        ''' 
        data = The data user want to compute in SVM, must be 2d-array
        label_pos = label column in data, must be integer and in last column
        '''
        try:
            self.label = data[:, label_pos].reshape(-1,1)
            self.data = data[:, :label_pos]
        except:
            print("\n===Error in trainer-init : data must be np 2d array or label_pos out of index===")
            return False
        
        # Swap data to disk to reduce memory limit
        try:
            np.save("rawData", data)
            np.save("tempLabel", self.label)
            np.save("tempData", self.data)
        except:
            print("\n===Error in trainer-init : can't save temp data===")
            return False
        
        self.delta = delta
        self.beta = beta
        self.r = r
        self.v = v
        self.rawData = "rawData.npy"
        self.tempLabel = "tempLabel.npy"
        self.tempData = "tempData.npy"

        self.rlabel = []
        self.rdata = []

        self.model = []
        
        # REDEFINE AS REQUIRED
        # r = ratio of data, v=kfolds, c=hyperparam, g=gamma, k=kernel (1=nonlinear), s=strategy(0=OvO, 1=OvR)
        self.handler = self.make() and self.tune(c=1, g=0.3, k=2, s=0, tildaA=None, degree=2, poly_c=1)


    def make(self):
        ''' 
        Split data to reduced set or validation set
        r : Ratio of data we want to use
        v : Number of folds for in Crossvalidation
        '''

        # self.setIndex is a dictionary which is composed of 'subset' and 'fold' , every fold has two part : 'train' and 'test'

        # Call original data back from disk
        if "self.label" not in locals() or "self.data" not in locals():
            try:
                self.label = np.load(self.tempLabel)
                self.data = np.load(self.tempData)
            except:
                print("\n===Error in trainer-make : can't load tempData and tempLabel===")
                return False

        self.ratio = self.r
        self.numFold = self.v

        try:
            # split_dict = splitData(y.reshape(-1,1), ratio=0.5, num_fold=5)
            self.setIndex = DataSplitter.splitData(self.label, self.ratio, self.numFold)
            self.rlabel = self.label[self.setIndex["subset"]].reshape(len(self.setIndex["subset"]), 1)
            self.rdata = self.data[self.setIndex["subset"], :]
        except:
            print("\n===Error in trainer-make : data can't be splited===")
            return False

        # Free object which don't need
        del self.label
        del self.data

        return True


    def tune(self, c, g, k, s, tildaA=None, degree=None, poly_c=None):
        '''' 
        Set the parameters
        c = Best important parameter of SVM, controling error
        g = 'Gamma' of rbf kernel
        k = kernel type, 0 -> linear, 1 -> nonlinear
        s = training strategy, 0 -> One-Against-One, 1 -> One-Against-Rest
        '''
        self.gamma = g
        self.C = c
        self.kernelType = k
        self.strategy = s
        self.tildaA = tildaA
        self.degree = degree
        self.poly_c = poly_c

        return True


    def multiClassifier(self, ith):
        ''' 
        For multi label dataset: 
        One-Against-One - if self.strategy = 0, 
        One-Against-Rest - if self.strategy = 1
        ith = the number of set in "CrossValidation
        
        Output:
        model = store w of multi classifier
        '''

        # Get unique label from reduced label set
        label_var = np.unique(self.rlabel)
        
        # Build kernel matrix for ith reduced dataset
        K = Kernel.buildKernel(
            self.kernelType,
            self.gamma,
            self.rdata[self.setIndex["fold"][ith]["train"], :],
            np.array([]),
            self.degree,
            self.poly_c)

        model = {}
        convergence_info = {}

        # One-Against-One
        # from sklearn.multiclass import OneVsOneClassifier
        # clf = OneVsOneClassifier(LinearSVC(dual="auto", random_state=0)).fit(X_train, y_train)
        # # clf = OneVsRestClassifier(SVC()).fit(X, y)
        # clf.predict(X_test[:10])
        if self.strategy == 0:
            if K["flag"] == "dual":
                for i in label_var:
                    model[i] = {}
                    # for j in label_var[np.where(label_var == i)[0] + 1 : label_var.shape[0]]: #0+1:
                    for index in np.where(label_var == i)[0]:
                        for j in label_var[index + 1:]:
                            model[i][j] = {}
                            var_pos = [y for y, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x == i]
                            var_neg = [y for y, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x == j]
                            
                            # From set catch two cluster with different label, give then positive and negative label, i -> A+, j -> A-
                            model[i][j]["col"] = var_pos + var_neg
                            ssvm = SSVM_Conv(K["Kernel"][var_pos, :][:, model[i][j]["col"]], K["Kernel"][var_neg, :][:, model[i][j]["col"]],
                                        self.C, np.ones((len(model[i][j]["col"]), 1)), np.array([0]), delta=self.delta, beta=self.beta)

                            # Store the model in i-j-label
                            model[i][j]["model"], convergence_info['info'] = ssvm.train()
                            del ssvm

            else:
                # Use A's all column because not dual form
                col = range(K["Kernel"].shape[1])
                for i in label_var:
                    model[i] = {}
                    for index in np.where(label_var == i)[0]:
                        for j in label_var[index + 1:]:
                            model[i][j] = {}
                            var_pos = [y for y, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x == i]
                            var_neg = [y for y, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x == j]
                            model[i][j]["col"] = col

                            ssvm = SSVM_Conv(K["Kernel"][var_pos, :],K["Kernel"][var_neg, :], self.C, np.zeros((len(col), 1)), np.array([0]), delta=self.delta, beta=self.beta)
                            model[i][j]["model"], convergence_info['info'] = ssvm.train()
                            del ssvm

        # One-Against-Rest
        elif self.strategy == 1:
            for var in label_var:
                var_pos = [i for i, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x == var]
                var_rest = [i for i, x in enumerate(self.rlabel[self.setIndex["fold"][ith]["train"]]) if x != var]
                ssvm = SSVM_Conv(K["Kernel"][var_pos, :], K["Kernel"][var_rest, :], self.C, np.zeros((K["Kernel"].shape[1], 1)), np.array([0]), delta=self.delta, beta=self.beta)
                model[var], convergence_info['info'] = ssvm.train()
                del ssvm

        else:
            print(
                "\n===Error in trainer-multiClassifier : self.strategy must equal to 0 or 1==="
            )
            return False
        return model, convergence_info


    def multiPredictor(self, ith, testIndex):
        ''' 
        ith = the number of set in "CrossValidation"
        testIndex = the index of testing data
        
        Output:
        result = the test result
        '''

        # Get unique label from reduced label set
        label_var = np.unique(self.rlabel)

        # Build kernel matrix for ith reduced dataset
        K = Kernel.buildKernel(
            self.kernelType,
            self.gamma,
            self.rdata[testIndex, :],
            self.rdata[self.setIndex["fold"][ith]["train"], :],
            self.degree,
            self.poly_c)
        
        # One-Against-One:
        if self.strategy == 0:
            result = np.zeros((len(testIndex), label_var.size + 1))
            for i in label_var:
                # for j in label_var[np.where(label_var == i)[0] + 1 : label_var.shape[0]]:
                for index in np.where(label_var == i)[0]:
                    for j in label_var[index + 1:]:
                        temp = (np.dot(K["Kernel"][:, self.model[ith][i][j]["col"]],self.model[ith][i][j]["model"]["w"],) 
                                - self.model[ith][i][j]["model"]["b"])

                        for k in range(len(testIndex)):
                            if temp[k] >= 0:
                                result[k, np.where(label_var == i)[0]] = (result[k, np.where(label_var == i)[0]] + 1)
                            else:
                                result[k, np.where(label_var == j)[0]] = (result[k, np.where(label_var == j)[0]] + 1)

            for i in range(len(testIndex)):
                result[i, label_var.size] = label_var[np.where(result[i, :] == np.max(result[i, :]))[0][0]] # max
            result = result[:, label_var.size].reshape(len(testIndex), 1)

        # One-Against-Rest
        elif self.strategy == 1:
            result = np.zeros((len(testIndex), 1))
            value = np.zeros((len(testIndex), 1))
            for var in label_var:
                temp = (np.dot(K["Kernel"], self.model[ith][var]["w"]) - self.model[ith][var]["b"])
                result[temp > value] = var

        else:
            print("\n===Error in trainer-multiPredictor : self.strategy must equal to 0 or 1===")
            return False
        return result

    
    def errorEstimate(self, comp_index, test_label):
        ''' 
        Check the result of test;
        testIndex = test data's index, use this to find the true label
        test = dataset be predicted by SSVM
        
        Output:
        return = "absolute error value"
        '''

        # error estimate
        compLabel = self.rlabel[comp_index]
        result = test_label == compLabel
        ErrV = 0
        for i in result:
            if i == False:
                ErrV = ErrV + 1

        # Confusion matrix
        conf_matrix = confusion_matrix(compLabel, test_label)        

        return float(ErrV) / compLabel.shape[0], conf_matrix


    def close(self):
        try:
            os.remove(self.rawData)
            os.remove(self.tempLabel)
            os.remove(self.tempData)
        except:
            print(
                "\n===Error in trainer-close : can't remove tempData and tempLabel==="
            )
            return False
        del self.rlabel
        del self.rdata
        del self.setIndex
        del self.model


    def train(self):
        ''''
        Can use two functions prior to training: "initial", "setParams" to adjust SVM
        
        Output:
        TErr = training error
        VErr = validation error
        '''

        self.model = []
        conv_info = {}
        if self.handler == True:
            start_t = time.time()
            TErr = np.zeros((self.numFold, 1))
            VErr = np.zeros((self.numFold, 1))
            con_matrices = []
            for ith in range(self.numFold):
                model, convergence_info = self.multiClassifier(ith)
                conv_info[f'model_{ith}'] = convergence_info
                self.model.append(model)
                result = self.multiPredictor(ith, self.setIndex["fold"][ith]["train"])
                TErr[ith], _ = self.errorEstimate(self.setIndex["fold"][ith]["train"], result)
                result = self.multiPredictor(ith, self.setIndex["fold"][ith]["test"])
                VErr[ith], conf_matrix_val = self.errorEstimate(self.setIndex["fold"][ith]["test"], result)
                con_matrices.append(conf_matrix_val)
            stop_t = time.time()

            # print("\nTraining accuracy:\n")
            # print(1 - TErr)
            # print("\nValidation accuracy:\n")
            # print(1 - VErr)
            # print("\nTraining model using %f s..." % (stop_t - start_t))

            result = {
                "TAcc": 1 - np.mean(TErr),
                "VAcc": 1 - np.mean(VErr),
                "time": stop_t - start_t,
                "confusion_matrix": con_matrices
            }
            self.close()
            return result, conv_info

        else:
            print("\n===Error in trainer-train : 'make' or 'tune' not correct===")
            return False